In [1]:
from google.colab import drive
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
from torch.utils.data import DataLoader, Dataset
from torch.nn import BCEWithLogitsLoss
import torch
import os

# Mount Google Drive
drive.mount('/content/drive')

# Navigate to the directory where your datasets are located
# Replace "Your_Folder" with the actual folder path
os.chdir('/content/drive/MyDrive/Colab Notebooks')

Mounted at /content/drive


In [2]:
# Load the labeled training dataset from a CSV file
train_dataset_path = 'train.csv'
df_train = pd.read_csv(train_dataset_path)

# Load the labeled test dataset from a CSV file
test_dataset_path = 'test.csv'
df_test = pd.read_csv(test_dataset_path)

# Split the training dataset into training and validation sets
train_df, val_df = train_test_split(df_train, test_size=0.2, random_state=42)


In [3]:
# Define a custom dataset class
class SentimentDataset(Dataset):
    def __init__(self, review, sentiment, tokenizer, max_length=128):
        self.review = review
        self.sentiment = sentiment
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.review)

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.review[idx],
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )

        input_ids = encoding['input_ids'].squeeze()
        attention_mask = encoding['attention_mask'].squeeze()

        # Convert string labels to integers
        label_mapping = {'positive': 1.0, 'negative': 0.0}
        labels = torch.tensor(label_mapping[self.sentiment[idx]])

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels
        }

In [4]:
print(df_train.columns)

Index(['review', 'sentiment'], dtype='object')


In [5]:
print(df_test.columns)

Index(['review', 'sentiment'], dtype='object')


In [6]:
# Define the model, tokenizer, and optimizer
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=1)
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
optimizer = AdamW(model.parameters(), lr=1e-5)

# Prepare the datasets
train_dataset = SentimentDataset(train_df['review'].tolist(), train_df['sentiment'].tolist(), tokenizer)
val_dataset = SentimentDataset(val_df['review'].tolist(), val_df['sentiment'].tolist(), tokenizer)

# Define data loaders
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [7]:
# Training loop
epochs = 3
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

loss_fn = BCEWithLogitsLoss()

for epoch in range(epochs):
    model.train()
    total_loss = 0

    for batch in train_loader:
        inputs = {k: v.to(device) for k, v in batch.items()}
        optimizer.zero_grad()

        input_ids = inputs['input_ids']
        attention_mask = inputs['attention_mask']
        labels = inputs['labels']

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    average_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{epochs} - Average Training Loss: {average_loss}")

    # Validation
    model.eval()
    val_loss = 0

    with torch.no_grad():
        for batch in val_loader:
            inputs = {k: v.to(device) for k, v in batch.items()}

            input_ids = inputs['input_ids']
            attention_mask = inputs['attention_mask']
            labels = inputs['labels']

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()

    average_val_loss = val_loss / len(val_loader)
    print(f"Epoch {epoch + 1}/{epochs} - Average Validation Loss: {average_val_loss}")

Epoch 1/3 - Average Training Loss: 0.11384538590240603
Epoch 1/3 - Average Validation Loss: 0.09562592327105812
Epoch 2/3 - Average Training Loss: 0.0728352264223504
Epoch 2/3 - Average Validation Loss: 0.09289659937362497
Epoch 3/3 - Average Training Loss: 0.04742221522332208
Epoch 3/3 - Average Validation Loss: 0.09377555695548653


In [8]:
# Save the fine-tuned model
model.save_pretrained('fine_tuned_distilbert-base-uncased_model')
tokenizer.save_pretrained('fine_tuned_distilbert-base-uncased_model')

('fine_tuned_distilbert-base-uncased_model/tokenizer_config.json',
 'fine_tuned_distilbert-base-uncased_model/special_tokens_map.json',
 'fine_tuned_distilbert-base-uncased_model/vocab.txt',
 'fine_tuned_distilbert-base-uncased_model/added_tokens.json',
 'fine_tuned_distilbert-base-uncased_model/tokenizer.json')